<a href="https://colab.research.google.com/github/lalva224/Market_Anomaly/blob/main/Market_Anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [ ]:
#open the file and try plotting each of the values when it is not an anomaly and when it is
df = pd.read_csv('/content/FinancialMarketData.xlsx - EWS.csv')
df.columns


Index(['Y', 'Data', 'XAU BGNL', 'ECSURPUS', 'BDIY', 'CRY', 'DXY', 'JPY', 'GBP',
       'Cl1', 'VIX', 'USGG30YR', 'GT10', 'USGG2YR', 'USGG3M', 'US0001M',
       'GTDEM30Y', 'GTDEM10Y', 'GTDEM2Y', 'EONIA', 'GTITL30YR', 'GTITL10YR',
       'GTITL2YR', 'GTJPY30YR', 'GTJPY10YR', 'GTJPY2YR', 'GTGBP30Y',
       'GTGBP20Y', 'GTGBP2Y', 'LUMSTRUU', 'LMBITR', 'LUACTRUU', 'LF98TRUU',
       'LG30TRUU', 'LP01TREU', 'EMUSTRUU', 'LF94TRUU', 'MXUS', 'MXEU', 'MXJP',
       'MXBR', 'MXRU', 'MXIN', 'MXCN'],
      dtype='object')

In [ ]:
target = df['Y']
features = df.drop(['Y','Data'],axis=1)
features = features.dropna()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=42)
scaler = StandardScaler()
#before scaling some calculations need to be done such as mean and SD, this is only done once in the training data.
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
#pickle allows us to save and load ML models
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score,classification_report
def evaluate_and_save_model(model,X_train,X_test,y_train,y_test,filename):
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test,y_pred)
  print(f"{model.__class__.__name__} Accuraccy:{accuracy: .4f}")
  print(f"\nClassification Report: \n {classification_report(y_test,y_pred)}")
  print('------------------------')

  #wb opens file in binary mode. We are just saving the model in a file. This way we can use the model later on
  with open(filename,"wb") as file:
    pickle.dump(model,file)

  print(f"Model saved as {filename}")

xgb_model = xgb.XGBClassifier(random_state=42)
evaluate_and_save_model(xgb_model,X_train,X_test,y_train,y_test,'xgb_model.pkl')

dt_model = DecisionTreeClassifier(random_state=42)
evaluate_and_save_model(dt_model,X_train,X_test,y_train,y_test,'dt_model.pkl')

rf_model = RandomForestClassifier(random_state=42)
evaluate_and_save_model(rf_model,X_train,X_test,y_train,y_test,'rf_model.pkl')

nb_model = GaussianNB()
evaluate_and_save_model(nb_model,X_train,X_test,y_train,y_test,'nb_model.pkl')

knn_model = KNeighborsClassifier()
evaluate_and_save_model(knn_model,X_train,X_test,y_train,y_test,'knn_model.pkl')

svm_model = SVC(random_state=42)
evaluate_and_save_model(svm_model,X_train,X_test,y_train,y_test,'svm_model.pkl')

XGBClassifier Accuraccy: 0.9058

Classification Report: 
               precision    recall  f1-score   support

           0       0.92      0.97      0.94       177
           1       0.84      0.67      0.75        46

    accuracy                           0.91       223
   macro avg       0.88      0.82      0.84       223
weighted avg       0.90      0.91      0.90       223

------------------------
Model saved as xgb_model.pkl
DecisionTreeClassifier Accuraccy: 0.8834

Classification Report: 
               precision    recall  f1-score   support

           0       0.92      0.94      0.93       177
           1       0.74      0.67      0.70        46

    accuracy                           0.88       223
   macro avg       0.83      0.81      0.82       223
weighted avg       0.88      0.88      0.88       223

------------------------
Model saved as dt_model.pkl
RandomForestClassifier Accuraccy: 0.9103

Classification Report: 
               precision    recall  f1-score   s

In [ ]:
print('Amount of non anomalies: ',len(df[df['Y']==0]))
print('Amount of anomalies: ',len(df[df['Y']==1]))

Amount of non anomalies:  874
Amount of anomalies:  237


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled,y_resampled = smote.fit_resample(X_train,y_train)
evaluate_and_save_model(xgb_model,X_resampled,X_test,y_resampled,y_test,'xgb_model.pkl')

XGBClassifier Accuraccy: 0.9103

Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.95      0.94       177
           1       0.80      0.76      0.78        46

    accuracy                           0.91       223
   macro avg       0.87      0.86      0.86       223
weighted avg       0.91      0.91      0.91       223

------------------------
Model saved as xgb_model.pkl


In [ ]:

# from sklearn.ensemble import StackingClassifier

# meta_model = xgb.XGBClassifier(random_state=42)
# base_models = [('rf',RandomForestClassifier(random_state=42)),('xgb',xgb.XGBClassifier(randomstate=42))]

# stacking_clf = StackingClassifier(estimators=base_models,final_estimator=meta_model,cv=5)
# evaluate_and_save_model(stacking_clf,X_resampled,X_test,y_resampled,y_test,'stacking_clf.pkl')

In [ ]:
feature_importances = xgb_model.feature_importances_
feature_names = features.columns
importance_df = {
    'Feature':feature_names,
    'Importance':feature_importances
}
importance_df = pd.DataFrame(importance_df)
importance_df = importance_df.sort_values(by='Importance',ascending=False)
importance_df


,Feature,Importance
8,VIX,0.155402
19,GTITL10YR,0.134396
16,GTDEM2Y,0.077942
20,GTITL2YR,0.060840
37,MXJP,0.037049
28,LMBITR,0.029590
36,MXEU,0.026744
31,LG30TRUU,0.025108
32,LP01TREU,0.025080
34,LF94TRUU,0.023202
